In [1]:
import importlib
import pandas as pd
import numpy as np
import re

In [2]:
from functions.get_mon_data import *
from functions.functions_dataframe import * 
from functions.functions_shared import *
from parameters.filters_lists import *
from parameters.route import *
from parameters.regs import *

In [ ]:
""" 
Для формування списку ЄДРПОУ навчальних закладів, які потрібно наносити на мапу, використані бази даних МОН, опубліковані у листопаді 2019 року на Порталі відкритих даних.

### Перелік закладів загальної середньої та дошкільної освіти (ЗЗСДО)
    https://data.gov.ua/dataset/bd70757a-ab49-46be-a650-608a9b3bd7b1
    
### Перелік закладів позашкільної освіти (державна форма власності)
    https://data.gov.ua/dataset/9a88fb96-a043-4769-acf1-1c4a948ec285
    Перелік закладів позашкільної освіти системи Міністерства освіти і науки України
    https://data.gov.ua/dataset/18d7fb70-3781-4017-88d2-651e14d14211
    Заклади професійної (професійно-технічної) освіти
    https://data.gov.ua/dataset/824513a9-aa5e-45c8-b45e-dced6909761a
"""

In [ ]:
"""
df_raw - база даних завершених закупівель 24 обласних центрів з 1 серпня 2016 до 31 грудня 2019.
В Донецькій області - Краматорськ. В Луганській - Сєвєродонецьк. 
Згерерована у Профейсійному модулі аналітики bipro.prozorro.
"ИдентификаторЛота" є унікальним ідентифікатором рядка.
"""  

In [344]:
df_raw = pandas.read_csv('data/TrainingProZorroForSchoolProject20162019.csv', sep = ';', low_memory = False)
training_data_columns = df_raw.columns.tolist()
print("Дані з файлу викачки BIPro Prozorro: " + "df_raw" + str(df_raw.shape))

Дані з файлу викачки BIPro Prozorro: df_raw(537893, 11)


##### Step 0: Підготовка бази МОН для використання у фільтруванні закупівель

In [4]:
expdata_sc = pandas.read_csv('data/mon_data/expdata_sc.csv', sep = ';', low_memory = False)
expdata_dnz = pandas.read_csv('data/mon_data/expdata_dnz.csv', sep = ';', low_memory = False)

In [5]:
expdata_sc = expdata_sc.fillna(0)
expdata_sc['edrpou'] = expdata_sc['edrpou'].astype(np.int64)
mask = (expdata_sc['edrpou'].astype(str).str.len() == 7) | (expdata_sc['edrpou'].astype(str).str.len() == 8)
expdata_sc = expdata_sc.loc[mask].sort_values(by=['edrpou'])
print("expdata_sc", expdata_sc.shape)

expdata_sc (15206, 21)


In [6]:
expdata_dnz['edrpou'] = pd.to_numeric(expdata_dnz['edrpou'],errors='coerce')
expdata_dnz = expdata_dnz.fillna(0)
expdata_dnz['edrpou'] = expdata_dnz['edrpou'].astype(np.int64)
mask = (expdata_dnz['edrpou'].astype(str).str.len() == 7) | (expdata_dnz['edrpou'].astype(str).str.len() == 8)
expdata_dnz = expdata_dnz.loc[mask].sort_values(by=['edrpou'])
print("expdata_dnz", expdata_dnz.shape)

expdata_dnz (15745, 21)


In [7]:
frames = [expdata_dnz, expdata_sc]
df_mon_data = pd.concat(frames)
print("База МОН шкіл і садків з 7-8 значинми ЄДРПОУ, df_mon_data", df_mon_data.shape)

База МОН шкіл і садків з 7-8 значинми ЄДРПОУ, df_mon_data (30951, 22)


In [8]:
filter_keywords_exclude = ["департамент", "управлін.*освіт", "відділ.*освіт"]
df_raw_upravlin_osvity = df_raw[df_raw['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_exclude)), case = False, na = False, regex = True)]
edrpou_upravlin_osvity_prozorro = df_raw_upravlin_osvity['IDOrganizator'].unique().tolist()
print("edrpou_upravlin_osvity_prozorro", len(edrpou_upravlin_osvity_prozorro))

mask = (df_mon_data['edrpou'].isin(edrpou_upravlin_osvity_prozorro))
df_mon_data_bez_upravlin_osvity = df_mon_data.loc[~mask]
print("df_mon_data_bez_upravlin_osvity", df_mon_data_bez_upravlin_osvity.shape)

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


edrpou_upravlin_osvity_prozorro 234
df_mon_data_bez_upravlin_osvity (30881, 22)


In [38]:
df_mon_data[df_mon_data['university_name'].str.contains('Дніпровський професійний залізничний ліцей')]

,edrpou,education_type_name,koatuu_full_name,koatuu_id,koatuu_id_u,koatuu_name,koatuu_name_u,language,location_type,post_index,...,university_address,university_address_u,university_director_fio,university_email,university_financing_type_name,university_level,university_name,university_phone,university_site,university_type_name


##### Step 1: Створити базу навчальних закладів, які самостійно купують через ProZorro. Виділення закупівель шкіл-замовників , згідно з базою МОН (пошук по колонці "IDOrganizator"

In [9]:
mon_data = df_mon_data_bez_upravlin_osvity.edrpou.unique().tolist()
# mon_data = get_mon_data()

df_tenders_by_schools_by_edrpou = df_raw[df_raw['IDOrganizator'].isin(mon_data)]
df_undefined_1 = df_raw[~df_raw['IDOrganizator'].isin(mon_data)]

print("Закупівлі шкіл-замовників, визначених через МОН: df_tenders_by_schools_by_edrpou " + str(df_tenders_by_schools_by_edrpou.shape))
print("Невизначені записи: df_undefined_1 " + str(df_undefined_1.shape))

Закупівлі шкіл-замовників, визначених через МОН: df_tenders_by_schools_by_edrpou (32188, 11)
Невизначені записи: df_undefined_1 (505705, 11)


In [10]:
# evaluation of results of Step 1

df_tenders_by_schools_by_edrpou['Организатор'].value_counts()

Загальноосвітній навчальний заклад "Київський спортивний ліцей-інтернат"                                                                                                                               542
КУ "Комунальна установа Сумська спеціалізована школа І-ІІІ ступенів №7 імені Максима Савченка Сумської міської ради"                                                                                   310
Навчально-виховний комплекс «Домінанта» (дошкільний навчальний заклад - спеціалізована школа І ступеня - суспільно-гуманітарна гімназія – СШ №204  з поглибленим вивченням фізичної культури)          300
Хмельницьке навчально-виховне об'єднання №28                                                                                                                                                           284
Муніципальний заклад вищої освіти "Київська Академія мистецтв"                                                                                                                              

##### Step 2: Виділення закупівель шкіл-замовників за допомогою пошуку ключових слів в колонці "Организатор" 

In [35]:
df_tenders_by_schools_by_keywords = df_undefined_1[df_undefined_1['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True) ]

print("Закупівлі шкіл-замовників, визначених через 'Организатор': df_tenders_by_schools_by_keywords " + str(df_tenders_by_schools_by_keywords.shape))

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


Закупівлі шкіл-замовників, визначених через 'Организатор': df_tenders_by_schools_by_keywords (9351, 11)


In [36]:
# evalutaion #1 of results of Step 2

df_tenders_by_schools_by_keywords['Организатор'].value_counts()

Централізована бухгалтерія дитячо-юнацьких спортивних шкіл                                                                                                            424
Державний професійно - технічний навчальний заклад "Дніпровський професійний залізничний ліцей"                                                                       391
Школа вищої спортивної майстерності міста Києва                                                                                                                       329
Київський професійний енергетичний ліцей                                                                                                                              275
Київська міська школа вищої спортивної майстерності                                                                                                                   232
Київський професійний ліцей транспорту                                                                                                                

In [60]:
# possible solution for Step 2

filter_keywords_step_2_exclude = ['професійн', 'міжшкільн']
df_tenders_by_schools_by_keywords_2 = df_tenders_by_schools_by_keywords[~df_tenders_by_schools_by_keywords['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_step_2_exclude)), 
            case = False, na = False, regex = True)]
print("Видалення із датафрейму df_tenders_by_schools_by_keywords так званих 'професійних' навчальних закладів", df_tenders_by_schools_by_keywords_2.shape)


filter_keywords_step_2_include = ["школа І-І.*ступенів", "навчальн.*виховн.*комплекс", "гімназ", "ліце"]
df_tenders_by_schools_by_keywords_2 = df_tenders_by_schools_by_keywords_2[df_tenders_by_schools_by_keywords_2['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_step_2_include)), 
            case = False, na = False, regex = True)]
print("Додаткове фільтрування датафрейму df_tenders_by_schools_by_keywords", df_tenders_by_schools_by_keywords_2.shape)

Видалення із датафрейму df_tenders_by_schools_by_keywords так званих 'професійних' навчальних закладів (6890, 11)
Додаткове фільтрування датафрейму df_tenders_by_schools_by_keywords (227, 11)


/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """
/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # This is added back by InteractiveShellApp.init_path()


In [61]:
# evalutaion #2 of results of Step 2

df_tenders_by_schools_by_keywords_2['Организатор'].value_counts()

Ліцей №2 м. Житомира                                                                                       51
Тернопільська загальноосвітня школа І-ІІІ ступенів №21                                                     48
Черкаська загальноосвітня школа І-ІІІ ступенів №26 ім. І.Ф.Момота                                          43
Житомирська загальноосвітня спеціалізована школа І-ІІІ ступенів №16                                        40
Навчально-виховний комплекс: загальноосвітня школа І-ІІІ ступенів-гімназія № 23 Вінницької міської ради    28
Державний навчальний заклад "Електрорадіотехнічний ліцей м.Полтави"                                         7
Миколаївська гімназія № 2 Миколаївської міської ради Миколаївської області                                  7
Тернопільський навчально-виховний комплекс «Школа-ліцей №6 ім. Н.Яремчука»                                  3
Name: Организатор, dtype: int64

##### Step 2.5: Об'єднання двох датафреймів шкіл-замовників в один

In [62]:
df_tenders_by_schools = pandas.concat([df_tenders_by_schools_by_edrpou, df_tenders_by_schools_by_keywords_2])

print("Закупівлі шкіл-замовників: df_tenders_by_schools " + str(df_tenders_by_schools.shape))
print("Невизначені записи: df_undefined_1 " + str(df_undefined_1.shape))

Закупівлі шкіл-замовників: df_tenders_by_schools (32415, 11)
Невизначені записи: df_undefined_1 (505705, 11)


##### Step 3: Класифікація шкіл-замовників по типах

In [63]:
columns_to_classify = ['Организатор']

df_tenders_by_schools_classified = classification_by_type_one_column(df_tenders_by_schools, columns_to_classify, filter_priority, filter_name_priority)

/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df['Тип закладу'] = "Невідомо"
/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df[column + "_check"] = ""
/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:40: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  case = False, na = False, regex = True)
/home/a

In [64]:
# analysis of Step 3

pd.set_option("display.max_colwidth", 200)
for_analysis_step_3 = df_tenders_by_schools_classified[df_tenders_by_schools_classified['Тип закладу'] == 'Невідомо'][['Организатор', 'Тип закладу']].sort_values(by=['Организатор'])
for_analysis_step_3['Организатор'].unique().tolist()

['Галицький коледж імені В’ячеслава Чорновола',
 'КУ ССШ №9',
 'Київський державний будинок художньої та технічної творчості',
 'МЕЛ№2 ММР МО',
 'Муніципальний заклад вищої освіти "Київська Академія мистецтв"',
 'ОЛІМПІЙСЬКИЙ КОЛЕДЖ ІМЕНІ ІВАНА ПІДДУБНОГО',
 'Рівненський міський Палац дітей та молоді',
 'Рівненський навчально-реабілітаційний центр "Особлива дитина" Рівненської міської ради',
 'Школа I-III ступенів №249 Деснянського району міста Києва']

##### Step 4: Нумерація шкіл-замовників

In [65]:
columns_to_numerate = ['Организатор']

df_tenders_by_schools_numerated = numeration_one_column(df_tenders_by_schools_classified, columns_to_numerate, reg_number)

/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df['Номер'] = given_df[column].str.findall(f"{regex}", flags = re.IGNORECASE).apply(''.join)


##### Step 4.5: Найменування шкіл-замовників

In [72]:
columns_to_name = ['Организатор']

df_tenders_by_schools_named = naming_one_column(df_tenders_by_schools_numerated, columns_to_name, reg_quotes)

In [73]:
# видалити закупівлі позашкільних закладів через фільтрування filter_poza

print('Було закупівель', df_tenders_by_schools_named.shape)
df_tenders_by_schools_named = df_tenders_by_schools_named[~df_tenders_by_schools_named['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_poza)),
                case = False, na = False, regex = True)]
print('Стало закупівель', df_tenders_by_schools_named.shape)

Було закупівель (32415, 15)


/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


Стало закупівель (31705, 15)


In [74]:
# evaluation of Step 4

# перетворити пусті строки в NaN
df_tenders_by_schools_named['Номер'] = df_tenders_by_schools_named['Номер'].replace(r'^\s*$', np.nan, regex=True)

for_analysis_step_4 = df_tenders_by_schools_named[df_tenders_by_schools_named['Назва'].isnull() & 
                                  df_tenders_by_schools_named['Номер'].isnull()]
print('Навчальні заклади, де не визначились номер чи назва:')
for_analysis_step_4['Организатор'].unique().tolist()

Навчальні заклади, де не визначились номер чи назва:


/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


['Тернопільська спеціальна загальноосвітня школа Тернопільської міської ради Тернопільської області',
 'Авіакосмічний ліцей ім. І.Сікорського НАУ м. Києва',
 'ОЛІМПІЙСЬКИЙ КОЛЕДЖ ІМЕНІ ІВАНА ПІДДУБНОГО',
 'Галицький коледж імені В’ячеслава Чорновола',
 'Харківська гімназія 116 Харківської міської ради Харківської області',
 'Перша міська гімназія Черкаської міської ради Черкаської області',
 'Івано-Франківський природничо-математичний ліцей Івано-Франківської міської ради Івано-Франківської області',
 'КОМУНАЛЬНА УСТАНОВА СУМСЬКА КЛАСИЧНА ГІМНАЗІЯ СУМСЬКОЇ МІСЬКОЇ РАДИ',
 'Львівська правнича гімназія',
 'КУ Хмельницький колегіум імені Володимира Козубняка',
 'Тернопільська спеціалізована загальноосвітня школа І ступеня повного дня з поглибленим вивченням іноземних мов Тернопільської міської ради Тернопільської області',
 'Рівненський міський Палац дітей та молоді',
 "Ліцей ім. В'ячеслава Чорновола Івано-Франківської міської ради",
 'Політехнічниий ліцей Національного технічного універс

##### Step 5: Виділення закупівель шкіл-не-замовників за допомогою пошуку ключових слів в колонках Тендер, ОписаниеТендера, Лот

In [81]:
filter_keywords_extended = ['шк[оі]л', 
                            'загальноосвітн', 
                            'гімназ', 
                            'ліце[йя]', 
                            'колегіум', 
                            'сад[ко][аку]', 'яс[ле][ал]',
                            'міжшкільн[ио].*комбінат',
                            'навчальн.*виховн.*комплекс',
                            'центр.*розвитк.*дитин',
                            'загальн.*середн'
                           ]
filter_keywords_strict_extended = ['зош', 'знз', 'сзш', '[ун]вк', 'днз', 'нво']

In [82]:
df_tenders_not_by_schools_by_keywords = df_undefined_1[df_undefined_1['Тендер'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_extended)), 
            case = False, na = False, regex = True)  | 
                                                      df_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_extended)), 
            case = False, na = False, regex = True) | 
                                                      df_undefined_1['Лот'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_extended)), 
            case = False, na = False, regex = True) | df_undefined_1['Тендер'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict_extended)), 
            case = False, na = False, regex = True) | df_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict_extended)), 
            case = False, na = False, regex = True) | df_undefined_1['Лот'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict_extended)), 
            case = False, na = False, regex = True)]

print("Закупівлі шкіл-не-замовників, визначених через 'Организатор': df_tenders_by_schools_by_keywords " + str(df_tenders_not_by_schools_by_keywords.shape))

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


Закупівлі шкіл-не-замовників, визначених через 'Организатор': df_tenders_by_schools_by_keywords (28708, 11)


In [83]:
# evaluation of Step 5

df_tenders_not_by_schools_by_keywords

,ИдентификаторЛота,Идентификатор,Организатор,Тендер,СуммаЛота,ОписаниеТендера,Лот,Адрес поставки,АдресОрганизатора,Главный орган,IDOrganizator
2,UA-2016-09-13-000354-a-L1,UA-2016-09-13-000354-a,ДЮСШ № 23,Придбання обладнання для медичного кабінету позашкільним навчальним закладом Деснянського раойну міста Києва на 2016 рік,17000,Обладнання для медичного кабінету позашкільного навчального закладу: кушетка - 1 шт.; письмовий стіл - 1 шт.; ширма; шафа канцелярська; медичний столик із скляною кришкою із засобами для надання н...,Придбання обладнання для медичного кабінету позашкільним навчальним закладом Деснянського раойну міста Києва на 2016 рік,"просп. Маяковського, 26-А","просп. Маяковського, 26-А",КМДА,34290400
158,UA-2016-12-20-002907-b-L1,UA-2016-12-20-002907-b,Департамент житлово-комунального господарства Чернівецької міської ради,Знесення та посадка дерев на Садгірському Центральному кладовищі на вул. Я.Налепки,73978,NaN,Знесення та посадка дерев на Садгірському Центральному кладовищі на вул. Я.Налепки,Героїв Майдану 176,"вул. Героїв Майдану,176",Чернівецька міська рада,25082708
159,UA-2016-12-14-000943-a-L1,UA-2016-12-14-000943-a,Управління освіти Солом'янської районної в місті Києві державної адміністрації,Капітальний ремонт місць загального користування в СЗШ №166,200000,Капітальний ремонт місць загального користування в СЗШ №166,Капітальний ремонт місць загального користування в СЗШ №166,"вул.Єреванська, 20","вулиця Пітерська, 12",КМДА,37485490
186,UA-2016-11-02-000470-a-L1,UA-2016-11-02-000470-a,Управління у справах фізичної культури і спорту Миколаївської міської ради,"поточнийремонтдушових приміщень спортивної школи розташованої за адресою: пл. Заводська, 1",67191.86,додаток до річного плану,"поточнийремонтдушових приміщень спортивної школи розташованої за адресою: пл. Заводська, 1","вул. Потьомкінська,95-А","вул. Потьомкінська,95-А",Миколаївська міська рада,2930482
248,UA-2016-11-02-000583-a-L1,UA-2016-11-02-000583-a,Управління у справах фізичної культури і спорту Миколаївської міської ради,"капітальний ремонт системи автоматичної пожежної сигналізації та сповіщення про пожежу в приміщеннях дитячо-юнацької спортивної школи №1 за адресою: м. Миколаїв, вул. Театральна, 41 А",151705.2,додаток до річного плану,"капітальний ремонт системи автоматичної пожежної сигналізації та сповіщення про пожежу в приміщеннях дитячо-юнацької спортивної школи №1 за адресою: м. Миколаїв, вул. Театральна, 41 А","вул. Потьомкінська,95-А","вул. Потьомкінська,95-А",Миколаївська міська рада,2930482
258,UA-2016-12-20-002973-b-L1,UA-2016-12-20-002973-b,Департамент житлово-комунального господарства Чернівецької міської ради,Капітальний ремонт світлофорного об'єкту на вул.Південно-Кільцевій - вул. С.Воробкевича - ЗНЗ №22 в м.Чернівцях,241312,NaN,Капітальний ремонт світлофорного об'єкту на вул.Південно-Кільцевій - вул. С.Воробкевича - ЗНЗ №22 в м.Чернівцях,Героїв Майдану 176,"вул. Героїв Майдану,176",Чернівецька міська рада,25082708
260,UA-2016-12-20-000859-b-L1,UA-2016-12-20-000859-b,Департамент житлово-комунального господарства Чернівецької міської ради,"Поточний ремонт гравійної дороги вул. І. Карбулицького вздовж огорожі НВК ""Берегиня"" в м . Чернівцях",79154,NaN,"Поточний ремонт гравійної дороги вул. І. Карбулицького вздовж огорожі НВК ""Берегиня"" в м . Чернівцях",Героїв Майдану 176,"вул. Героїв Майдану,176",Чернівецька міська рада,25082708
262,UA-2016-12-29-000877-c-L1,UA-2016-12-29-000877-c,Управління освіти Солом'янської районної в місті Києві державної адміністрації,Поточний (аваріний) ремонт запчастин МІТП в ДНЗ №650 та ДНЗ №649,57178,Поточний (аваріний) ремонт запчастин МІТП в ДНЗ №650,Поточний (аваріний) ремонт запчастин МІТП в ДНЗ №650 та ДНЗ №649,"вул. Соломянська, 19-а","вулиця Пітерська, 12",КМДА,37485490
286,UA-2016-09-19-000183-a-L1,UA-2016-09-19-000183-a,Управління освіти Солом'янської районної в місті Києві державної адміністрації,Поточний ремонт котельні дитячого садку №211,2990,Поточний ремонт котельні дитячого садку №211

##### Step 6: Класифікація шкіл-не-замовників по типах

In [93]:
# тимчасово виніс фільтри сюди для тестування порядковості

filter_priority = [
    filter_nvk,
    filter_gimnasium,
    filter_liceum,
    filter_specializovana,
    filter_internat,
    filter_kombinat,
    filter_sanatorna,
    filter_specializovana,
    filter_specialna,
    filter_sadik,
    filter_shkola
]
filter_name_priority = [
    filter_nvk_name,
    filter_gimnasium_name,
    filter_liceum_name,
    filter_specializovana_name,
    filter_internat_name,
    filter_kombinat_name,
    filter_sanatorna_name,
    filter_specializovana_name,
    filter_specialna_name,
    filter_sadik_name,
    filter_shkola_name
]

def classification_by_type_three_cols(df, columns, priority, names):
    given_df = df[:]
    checked_priority = priority[::-1]
    checked_names = names[::-1]
    given_df['Тип закладу'] = "Невідомо"

    for column in columns:
        given_df[column + "_check"] = ""

    for idx in range(len(checked_priority)):
        checked_filter = checked_priority[idx]
        for column in columns:
            given_df[column + "_check"] = given_df[column].astype(str).str.contains(r'({})'.format('|'.join(checked_filter)),
                case = False, na = False, regex = True)

        checked_name = checked_names[idx]
        given_df['Тип закладу'] = np.where((given_df['Тендер_check'] == True) | (given_df['ОписаниеТендера_check'] == True) | (given_df['Лот_check'] == True), checked_name, given_df['Тип закладу'])
    return given_df

In [214]:
columns_to_classify = ['Тендер', 'ОписаниеТендера', 'Лот']

df_tenders_not_by_schools_classified = classification_by_type_three_cols(df_tenders_not_by_schools_by_keywords, columns_to_classify, filter_priority, filter_name_priority)

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [215]:
df_tenders_not_by_schools_classified['Тип закладу'].value_counts()

ДНЗ                                          12238
Загальноосвітня школа                         5333
Невідомо                                      5013
Навчально-виховний комплекс (об'єднання)      3095
Гімназія                                      1298
Спеціалізована школа                          1137
Ліцей                                          232
Спеціальна загальноосвітня школа               127
Інтернат                                       105
Загальносвітня санаторна школа                  68
Міжшкільний навчально-виробничий комбінат       62
Name: Тип закладу, dtype: int64

In [98]:
# analysis of Step 6

pd.set_option("display.max_colwidth", 200)
for_analysis_step_3 = df_tenders_not_by_schools_classified[df_tenders_not_by_schools_classified['Тип закладу'] == "Інтернат"][['Тендер', 'ОписаниеТендера', 'Лот', 'Тип закладу']].sort_values(by=['Тип закладу'])
for_analysis_step_3

,Тендер,ОписаниеТендера,Лот,Тип закладу
2171,Оплата послуг з проведення поточного ремонту електричних мереж школи-інтернат № 19,згідно ЄЗС ДК 021:2015 - 50710000-5 «Послуги з ремонту і технічного обслуговування електричного і механічного устаткування будівель» (Оплата послуг з проведення поточного ремонту електричних мереж...,Оплата послуг з проведення поточного ремонту електричних мереж школи-інтернат № 19,Інтернат
259110,"«ДСТУ Б.Д. 1.1-1:2013 Здійснення технічного нагляду за якістю виконуваних робіт по об’єкту: капітальний ремонт фасаду (заміна вікон) в школі-інтернаті № 17 за адресою: вул. Ушинського, 15 Солом’ян...","«ДСТУ Б.Д. 1.1-1:2013 Здійснення технічного нагляду за якістю виконуваних робіт по об’єкту: капітальний ремонт фасаду (заміна вікон) в школі-інтернаті № 17 за адресою: вул. Ушинського, 15 Солом’ян...","«ДСТУ Б.Д. 1.1-1:2013 Здійснення технічного нагляду за якістю виконуваних робіт по об’єкту: капітальний ремонт фасаду (заміна вікон) в школі-інтернаті № 17 за адресою: вул. Ушинського, 15 Солом’ян...",Інтернат
259096,"«ДСТУ Б.Д. 1.1-1:2013 Здійснення технічного нагляду за якістю виконуваних робіт по об’єкту: капітальний ремонт фасаду (заміна вікон) в школі-інтернаті № 13 за адресою: вул. Новопольова, 106 Солом’...","«ДСТУ Б.Д. 1.1-1:2013 Здійснення технічного нагляду за якістю виконуваних робіт по об’єкту: капітальний ремонт фасаду (заміна вікон) в школі-інтернаті № 13 за адресою: вул. Новопольова, 106 Солом’...","«ДСТУ Б.Д. 1.1-1:2013 Здійснення технічного нагляду за якістю виконуваних робіт по об’єкту: капітальний ремонт фасаду (заміна вікон) в школі-інтернаті № 13 за адресою: вул. Новопольова, 106 Солом’...",Інтернат
258741,"ДК 021:2015 45220000-5 - Інженерні та будівельні роботи ( з розробки проектно-кошторисної документації робочого проекту автоматичної установки пожежної сигналізації, системи оповіщення людей про п...",NaN,"ДК 021:2015 45220000-5 - Інженерні та будівельні роботи ( з розробки проектно-кошторисної документації робочого проекту автоматичної установки пожежної сигналізації, системи оповіщення людей про п...",Інтернат
258727,ДК 021:2015 45220000-5 - Інженерні та будівельні роботи ( з капітального ремонту Одеської загальноосвітньої школи -інтернат №5 І-ІІІ ступенів Одеської міської ради Одеської області за адресою: м.О...,NaN,ДК 021:2015 45220000-5 - Інженерні та будівельні роботи ( з капітального ремонту Одеської загальноосвітньої школи -інтернат №5 І-ІІІ ступенів Одеської міської ради Одеської області за адресою: м.О...,Інтернат
254346,"ДБН А.2.2-3:2014 «Поточний (аварійний) ремонт місць загального користування в школі-інтернаті № 17 за адресою: вул. Ушинського, 15 у Солом’янському р-ні м. Києва (за кодом CPV: 45450000-6 Інші зав...","ДБН А.2.2-3:2014 «Поточний (аварійний) ремонт місць загального користування в школі-інтернаті № 17 за адресою: вул. Ушинського, 15 у Солом’янському р-ні м. Києва (за кодом CPV: 45450000-6 Інші зав...","ДБН А.2.2-3:2014 «Поточний (аварійний) ремонт місць загального користування в школі-інтернаті № 17 за адресою: вул. Ушинського, 15 у Солом’янському р-ні м. Києва (за кодом CPV: 45450000-6 Інші зав...",Інтернат
249709,"М'який інвентар в школу-інтернат № 8, Шевченківського району м. Києва","М'який інвентар в школу-інтернат № 8, Шевченківського району м. Києва","М'який інвентар в школу-інтернат № 8, Шевченківського району м. Києва",Інтернат
247536,"ДК 021: 2015 код 50410000-2 «Послуги з ремонту і технічного обслуговування вимірювальних, випробувальних і контрольних приладів» (послуги з ремонту і технічного обслуговування протипожежного облад...","ДК 021: 2015 код 50410000-2 «Послуги з ремонту і технічного обслуговування вимірювальних, випробувальних і контрольних приладів» (послуги з ремонту і технічного обслуговування протипожежного облад...","ДК 021: 2015 код 50410000-2 «Послуги з ремонту і технічного обслуговування вимірювальних, випробувальних і контрольних приладів» (послуги з ремонту і технічного обслуговування протипожежного облад...",Інтернат
245167,"Спеціаль

##### Step 7: Нумерація шкіл-не-замовників

In [389]:
def numeration(given_df, columns_to_classify, regex):
    for column in columns_to_classify:
        given_df[column + action_mark] = given_df[column].str.findall(f"{regex}", flags = re.IGNORECASE)
        given_df[column + '_' + action_mark] = [','.join(map(str, l)) for l in given_df[column + action_mark]]
        given_df = given_df.drop([column + action_mark], axis=1)
        given_df[column + '_' + action_mark + '_count'] = given_df[column + '_' + action_mark].str.count('№')
    return given_df

In [406]:
columns_to_numerate_step7 = ['Тендер', 'Лот']
step_mark = 'step7'
action_mark = 'numeration'
df_step7 = df_tenders_not_by_schools_classified[df_tenders_not_by_schools_classified['Тип закладу'] != 'Невідомо'].copy()

df_step7_numerated = numeration(df_step7, columns_to_numerate_step7, reg_number)
df_step7_numerated['number'] = np.where(((df_step7_numerated['Тендер_numeration_count'] == 1) | 
                                        (df_step7_numerated['Лот_numeration_count'] == 1)) & 
                                        (df_step7_numerated['Лот_numeration_count'] == df_step7_numerated['Тендер_numeration_count']), 
                                        df_step7_numerated['Лот_numeration'], 
                                        'Невідомо')

In [392]:
df_step7_numerated_undefined = df_step7_numerated[df_step7_numerated['number'] == 'Невідомо']
df_step7_numerated = df_step7_numerated[df_step7_numerated['number'] != 'Невідомо']

df_step7_numerated_undefined['number'] = np.where(df_step7_numerated_undefined['Лот_numeration_count'] == 1, 
                                                 df_step7_numerated_undefined['Лот_numeration'], 
                                                 'Невідомо')

In [393]:
df_step7_numerated_undefined_2 = df_step7_numerated_undefined[df_step7_numerated_undefined['number'] == 'Невідомо']
df_step7_numerated_2 = df_step7_numerated_undefined[df_step7_numerated_undefined['number'] != 'Невідомо']

df_step7_numerated_undefined_2['number'] = np.where(df_step7_numerated_undefined_2['Тендер_numeration_count'] == 1, 
                                                 df_step7_numerated_undefined_2['Тендер_numeration'], 
                                                 'Невідомо')

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [394]:
df_undefined_2 = df_step7_numerated_undefined_2[df_step7_numerated_undefined_2['number'] == 'Невідомо']
print("df_undefined_2", df_undefined_2.shape)

df_step7_numerated_3 = df_step7_numerated_undefined_2[df_step7_numerated_undefined_2['number'] != 'Невідомо']

df_undefined_2 (8593, 20)


In [395]:
frames = [df_step7_numerated, df_step7_numerated_2, df_step7_numerated_3]
df_tenders_not_by_schools_numerated = pd.concat(frames)

In [396]:
col_names_step7 = training_data_columns.copy()
col_names_step7 += ['number', 'Тип закладу']

df_tenders_not_by_schools_numerated = df_tenders_not_by_schools_numerated[col_names_step7]
print("df_tenders_not_by_schools_numerated", df_tenders_not_by_schools_numerated.shape)

df_tenders_not_by_schools_numerated (20115, 13)


##### Step 8: Найменування шкіл-не-замовників

In [397]:
def naming(given_df, columns_to_classify, regex):
    for column in columns_to_classify:
        given_df[column + '_' + action_mark] = given_df[column].str.findall(f"{regex}", flags = re.IGNORECASE).str[0].str[-1]
    return given_df

In [398]:
columns_to_classify_step8 = ['Тендер', 'Лот']
action_mark = 'naming'
df_tenders_not_by_schools_named = df_undefined_2.copy()

df_named = naming(df_tenders_not_by_schools_named, columns_to_classify_step8, reg_quotes)

In [399]:
df_named['number'] = np.where((df_named['Тендер_naming'].notnull()) & (df_named['Тендер_naming'] != ''),
                             df_named['Тендер_naming'], 
                             'Невідомо')

df_undefined_3 = df_named[df_named['number'] == 'Невідомо']
df_tenders_not_by_schools_named = df_named[(df_named['number'] != 'Невідомо') & 
                                           (df_named['number'] != '') ]

In [400]:
df_tenders_not_by_schools_named = df_tenders_not_by_schools_named[col_names_step7]
print("df_tenders_not_by_schools_named", df_tenders_not_by_schools_named.shape)

df_tenders_not_by_schools_named (1013, 13)


##### Step 9: Об'єднання двох датафреймів шкіл-не-замовників

In [401]:
frames = [df_tenders_not_by_schools_numerated, df_tenders_not_by_schools_named]
df_tenders_not_by_schools = pd.concat(frames)

In [410]:
df_tenders_not_by_schools

,ИдентификаторЛота,Идентификатор,Организатор,Тендер,СуммаЛота,ОписаниеТендера,Лот,Адрес поставки,АдресОрганизатора,Главный орган,IDOrganizator,number,Тип закладу
159,UA-2016-12-14-000943-a-L1,UA-2016-12-14-000943-a,Управління освіти Солом'янської районної в місті Києві державної адміністрації,Капітальний ремонт місць загального користування в СЗШ №166,200000,Капітальний ремонт місць загального користування в СЗШ №166,Капітальний ремонт місць загального користування в СЗШ №166,"вул.Єреванська, 20","вулиця Пітерська, 12",КМДА,37485490,№166,Загальноосвітня школа
248,UA-2016-11-02-000583-a-L1,UA-2016-11-02-000583-a,Управління у справах фізичної культури і спорту Миколаївської міської ради,"капітальний ремонт системи автоматичної пожежної сигналізації та сповіщення про пожежу в приміщеннях дитячо-юнацької спортивної школи №1 за адресою: м. Миколаїв, вул. Театральна, 41 А",151705.2,додаток до річного плану,"капітальний ремонт системи автоматичної пожежної сигналізації та сповіщення про пожежу в приміщеннях дитячо-юнацької спортивної школи №1 за адресою: м. Миколаїв, вул. Театральна, 41 А","вул. Потьомкінська,95-А","вул. Потьомкінська,95-А",Миколаївська міська рада,2930482,№1,Невідомо
258,UA-2016-12-20-002973-b-L1,UA-2016-12-20-002973-b,Департамент житлово-комунального господарства Чернівецької міської ради,Капітальний ремонт світлофорного об'єкту на вул.Південно-Кільцевій - вул. С.Воробкевича - ЗНЗ №22 в м.Чернівцях,241312,NaN,Капітальний ремонт світлофорного об'єкту на вул.Південно-Кільцевій - вул. С.Воробкевича - ЗНЗ №22 в м.Чернівцях,Героїв Майдану 176,"вул. Героїв Майдану,176",Чернівецька міська рада,25082708,№22,Невідомо
286,UA-2016-09-19-000183-a-L1,UA-2016-09-19-000183-a,Управління освіти Солом'янської районної в місті Києві державної адміністрації,Поточний ремонт котельні дитячого садку №211,2990,Поточний ремонт котельні дитячого садку №211,Поточний ремонт котельні дитячого садку №211,"Вул. Леніна, 167","вулиця Пітерська, 12",КМДА,37485490,№211,Невідомо
297,UA-2016-12-29-000878-c-L1,UA-2016-12-29-000878-c,Управління освіти Солом'янської районної в місті Києві державної адміністрації,Поточний (аварійний) ремонт електрощитової з заміною лічильника в гімназії №177,59049.6,Поточний (аварійний) ремонт електрощитової з заміною лічильника в гімназії №177,Поточний (аварійний) ремонт електрощитової з заміною лічильника в гімназії №177,"вул. Курська, 12","вулиця Пітерська, 12",КМДА,37485490,№177,Гімназія
1006,UA-2017-04-28-001555-b-L1,UA-2017-04-28-001555-b,Управління освіти Солом'янської районної в місті Києві державної адміністрації,"ДСТУ Б.Д. 1.1-1:2013 Капітальний ремонт інженерних мереж в СЗШ № 12 за адресою: вул. Зелена, буд. 12 Солом’янського району м. Києва (за кодом CPV 45450000-6 Інші завершальні будівельні роботи)",306000,"ДСТУ Б.Д. 1.1-1:2013 Капітальний ремонт інженерних мереж в СЗШ № 12 за адресою: вул. Зелена, буд. 12 Солом’янського району м. Києва (за кодом CPV 45450000-6 Інші завершальні будівельні роботи)","ДСТУ Б.Д. 1.1-1:2013 Капітальний ремонт інженерних мереж в СЗШ № 12 за адресою: вул. Зелена, буд. 12 Солом’янського району м. Києва (за кодом CPV 45450000-6 Інші завершальні будівельні роботи)","вул. Зелена, буд. 12","вулиця Пітерська, 12",КМДА,37485490,№ 12,Загальноосвітня школа
1070,UA-2017-06-02-001017-a-L1,UA-2017-06-02-001017-a,ВК Управління освіти виконавчого комітету Полтавської міської ради,"Капітальний ремонт приміщення харчоблоку Полтавського СНВК № 45 Полтавської міської ради по вул. Грушевського, 17а",320383.2,NaN,"Капітальний ремонт приміщення харчоблоку Полтавського СНВК № 45 Полтавської міської ради по вул. Грушевського, 17а","вул. Грушевського, 17а","вул. Соборності, 36",Полтавська міська рада,2145725,№ 45,Навчально-виховний комплекс (об'єднання)
1096,UA-2017-04-28-000337-c-L1,UA-2017-04-28-000337-c,Управління освіти Деснянської районної в місті Києві державної адміністрації,"Капітальний ремонт покрівлі в дошкільному навчальному закладі (ясла-садок) № 534 Деснянського району міста Києва, вул. Марша

In [409]:
df_tenders_by_schools_named

,ИдентификаторЛота,Идентификатор,Организатор,Тендер,СуммаЛота,ОписаниеТендера,Лот,Адрес поставки,АдресОрганизатора,Главный орган,IDOrganizator,Тип закладу,Организатор_check,Номер,Назва
10,UA-2016-11-24-001328-c-L1,UA-2016-11-24-001328-c,Тернопільська спеціальна загальноосвітня школа Тернопільської міської ради Тернопільської області,Персональний комп'ютер,20000,Персональний комп'ютер,Персональний комп'ютер,"вул. Академіка Сахарова, 4","вул. Сахарова,4",Тернопільська міська рада,14040606,Загальноосвітня школа,True,NaN,NaN
24,UA-2016-11-29-001070-c-L1,UA-2016-11-29-001070-c,Тернопільська спеціальна загальноосвітня школа Тернопільської міської ради Тернопільської області,Капітальний ремонт,38933,Капітальний ремонт,Капітальний ремонт,"вул. Академіка Сахарова, 4","вул. Сахарова,4",Тернопільська міська рада,14040606,Загальноосвітня школа,True,NaN,NaN
26,UA-2016-12-28-000118-c-L1,UA-2016-12-28-000118-c,ХМЕЛЬНИЦЬКА СПЕЦІАЛІЗОВАНА ЗАГАЛЬНООСВІТНЯ ШКОЛА №19 I-III СТУПЕНІВ ІМЕНІ АКАДЕМІКА МИХАЙЛА ПАВЛОВСЬКОГО,комп'ютерна техніка,154500,NaN,комп'ютерна техніка,"Кам'янецька,164","ВУЛИЦЯ КАМ'ЯНЕЦЬКА, будинок 164",Хмельницька міська рада,22764070,Загальноосвітня школа,True,№19,NaN
28,UA-2016-12-29-000048-c-L1,UA-2016-12-29-000048-c,"КОМУНАЛЬНИЙ ЗАКЛАД ""ЛУЦЬКИЙ ЛІЦЕЙ ЛУЦЬКОЇ МІСЬКОЇ РАДИ ВОЛИНСЬКОЇ ОБЛАСТІ""",Багатофункціональний пристрій,199000,"БФПCanon IR-2204, БФП Canon i-SENSYS MF217W",Багатофункціональний пристрій,"Дубнівська,32","Дубнівська, 32",Луцька міська рада,21733534,Загальноосвітня школа,True,NaN,ЛУЦЬКИЙ ЛІЦЕЙ ЛУЦЬКОЇ МІСЬКОЇ РАДИ ВОЛИНСЬКОЇ ОБЛАСТІ
37,UA-2016-12-29-000053-c-L1,UA-2016-12-29-000053-c,"КОМУНАЛЬНИЙ ЗАКЛАД ""ЛУЦЬКИЙ ЛІЦЕЙ ЛУЦЬКОЇ МІСЬКОЇ РАДИ ВОЛИНСЬКОЇ ОБЛАСТІ""",Комп'ютери,199900,"Комп'ютер Intel G1840/4G ОЗУ/500G HDD/LG 19M38A-B, Комп'ютер Intel G3250/4G ОЗУ/500G HDD/DWD-RW/LG 22M38A-B, ноутбук Lenovo IdeaPad 100(80MJ0041UA)15'6",Комп'ютери,"Дубнівська,32","Дубнівська, 32",Луцька міська рада,21733534,Загальноосвітня школа,True,NaN,ЛУЦЬКИЙ ЛІЦЕЙ ЛУЦЬКОЇ МІСЬКОЇ РАДИ ВОЛИНСЬКОЇ ОБЛАСТІ
38,UA-2016-12-21-001132-b-L1,UA-2016-12-21-001132-b,Комунальна установа Сумська загальноосвітня школа І-ІІІ ступенів №18 Сумської міської ради,Комп'ютери,128100,NaN,Комп'ютери,"вул.Леваневського,8","вул. Леваневського, 8",Сумська міська рада,21117924,Загальноосвітня школа,True,№18,NaN
47,UA-2016-12-01-001153-c-L1,UA-2016-12-01-001153-c,Тернопільська спеціальна загальноосвітня школа Тернопільської міської ради Тернопільської області,Капітальний ремонт приміщень,37668,Капітальний ремонт приміщень,Капітальний ремонт приміщень,"вул.Сахарова,4","вул. Сахарова,4",Тернопільська міська рада,14040606,Загальноосвітня школа,True,NaN,NaN
48,UA-2016-12-19-000896-c-L1,UA-2016-12-19-000896-c,Тернопільська спеціальна загальноосвітня школа Тернопільської міської ради Тернопільської області,Капітальний ремонт приміщень,3399,Капітальний ремонт приміщень ТСЗОШ,Капітальний ремонт приміщень,"вул.Сахарова,4","вул. Сахарова,4",Тернопільська міська рада,14040606,Загальноосвітня школа,True,NaN,NaN
54,UA-2016-11-14-000900-b-L1,UA-2016-11-14-000900-b,Тернопільська спеціальна загальноосвітня школа Тернопільської міської ради Тернопільської області,Звіт про укладений договір,37000,Капітальний ремонт покрівлі,Звіт про укладений договір,"вул. Академіка Сахарова, 4","вул. Сахарова,4",Тернопільська міська рада,14040606,Загальноосвітня школа,True,NaN,NaN
64,UA-2016-12-06-000957-b-L1,UA-2016-12-06-000957-b,Тернопільська загальноосвітня школа І-ІІІ ступенів № 26 імені Дмитра Заплітного Тернопільської міської ради Тернопільської області,Поточний ремонт (заміна вікон),31440,Поточний ремонт (заміна вікон),Поточний ремонт (заміна вікон),"бульвар П. Куліша, 9","бульвар П.Куліша, 9",Тернопільська міська рада,14039951,Загальноосвітня школа,True,№ 26,NaN


In [152]:
columns_to_numerate = ['Тендер']

df_tenders_not_by_schools_numerated = numeration_one_column(df_tenders_not_by_schools_classified, columns_to_numerate, reg_number)

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [153]:
columns_to_numerate = ['Лот']

df_tenders_not_by_schools_numerated_2 = df_tenders_not_by_schools_numerated[df_tenders_not_by_schools_numerated['Номер'] == '']
df_tenders_not_by_schools_numerated_2 = numeration_one_column(df_tenders_not_by_schools_numerated_2, columns_to_numerate, reg_number)

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [154]:
columns_to_numerate = ['ОписаниеТендера']

df_tenders_not_by_schools_numerated_3 = df_tenders_not_by_schools_numerated_2[df_tenders_not_by_schools_numerated_2['Номер'] == '']
df_tenders_not_by_schools_numerated_3['ОписаниеТендера'] = df_tenders_not_by_schools_numerated_3['ОписаниеТендера'].fillna('')
df_tenders_not_by_schools_numerated_3 = numeration_one_column(df_tenders_not_by_schools_numerated_3, columns_to_numerate, reg_number)

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [156]:
df_tenders_not_by_schools_not_numerated = df_tenders_not_by_schools_numerated_3[df_tenders_not_by_schools_numerated_3['Номер'] == '']

In [158]:
df_tenders_not_by_schools_numerated = df_tenders_not_by_schools_numerated[df_tenders_not_by_schools_numerated['Номер'] != '']
df_tenders_not_by_schools_numerated_2 = df_tenders_not_by_schools_numerated_2[df_tenders_not_by_schools_numerated_2['Номер'] != '']
df_tenders_not_by_schools_numerated_3 = df_tenders_not_by_schools_numerated_3[df_tenders_not_by_schools_numerated_3['Номер'] != '']

In [160]:
frames = [df_tenders_not_by_schools_numerated, df_tenders_not_by_schools_numerated_2, df_tenders_not_by_schools_numerated_3]
df_tenders_not_by_schools_numerated_all = pd.concat(frames)

In [161]:
df_tenders_not_by_schools_not_numerated.shape

(5371, 16)

In [164]:
df_tenders_not_by_schools_numerated_all.shape

(23337, 16)

In [193]:
df_tenders_not_by_schools_numerated_all

,ИдентификаторЛота,Идентификатор,Организатор,Тендер,СуммаЛота,ОписаниеТендера,Лот,Адрес поставки,АдресОрганизатора,Главный орган,IDOrganizator,Тип закладу,Тендер_check,ОписаниеТендера_check,Лот_check,Номер
159,UA-2016-12-14-000943-a-L1,UA-2016-12-14-000943-a,Управління освіти Солом'янської районної в місті Києві державної адміністрації,Капітальний ремонт місць загального користування в СЗШ №166,200000,Капітальний ремонт місць загального користування в СЗШ №166,Капітальний ремонт місць загального користування в СЗШ №166,"вул.Єреванська, 20","вулиця Пітерська, 12",КМДА,37485490,Загальноосвітня школа,False,False,False,№166
248,UA-2016-11-02-000583-a-L1,UA-2016-11-02-000583-a,Управління у справах фізичної культури і спорту Миколаївської міської ради,"капітальний ремонт системи автоматичної пожежної сигналізації та сповіщення про пожежу в приміщеннях дитячо-юнацької спортивної школи №1 за адресою: м. Миколаїв, вул. Театральна, 41 А",151705.2,додаток до річного плану,"капітальний ремонт системи автоматичної пожежної сигналізації та сповіщення про пожежу в приміщеннях дитячо-юнацької спортивної школи №1 за адресою: м. Миколаїв, вул. Театральна, 41 А","вул. Потьомкінська,95-А","вул. Потьомкінська,95-А",Миколаївська міська рада,2930482,Невідомо,False,False,False,№1
258,UA-2016-12-20-002973-b-L1,UA-2016-12-20-002973-b,Департамент житлово-комунального господарства Чернівецької міської ради,Капітальний ремонт світлофорного об'єкту на вул.Південно-Кільцевій - вул. С.Воробкевича - ЗНЗ №22 в м.Чернівцях,241312,NaN,Капітальний ремонт світлофорного об'єкту на вул.Південно-Кільцевій - вул. С.Воробкевича - ЗНЗ №22 в м.Чернівцях,Героїв Майдану 176,"вул. Героїв Майдану,176",Чернівецька міська рада,25082708,Невідомо,False,False,False,№22
262,UA-2016-12-29-000877-c-L1,UA-2016-12-29-000877-c,Управління освіти Солом'янської районної в місті Києві державної адміністрації,Поточний (аваріний) ремонт запчастин МІТП в ДНЗ №650 та ДНЗ №649,57178,Поточний (аваріний) ремонт запчастин МІТП в ДНЗ №650,Поточний (аваріний) ремонт запчастин МІТП в ДНЗ №650 та ДНЗ №649,"вул. Соломянська, 19-а","вулиця Пітерська, 12",КМДА,37485490,ДНЗ,False,False,False,№650№649
286,UA-2016-09-19-000183-a-L1,UA-2016-09-19-000183-a,Управління освіти Солом'янської районної в місті Києві державної адміністрації,Поточний ремонт котельні дитячого садку №211,2990,Поточний ремонт котельні дитячого садку №211,Поточний ремонт котельні дитячого садку №211,"Вул. Леніна, 167","вулиця Пітерська, 12",КМДА,37485490,Невідомо,False,False,False,№211
297,UA-2016-12-29-000878-c-L1,UA-2016-12-29-000878-c,Управління освіти Солом'янської районної в місті Києві державної адміністрації,Поточний (аварійний) ремонт електрощитової з заміною лічильника в гімназії №177,59049.6,Поточний (аварійний) ремонт електрощитової з заміною лічильника в гімназії №177,Поточний (аварійний) ремонт електрощитової з заміною лічильника в гімназії №177,"вул. Курська, 12","вулиця Пітерська, 12",КМДА,37485490,Гімназія,False,False,False,№177
789,UA-2017-01-19-001227-a-L2,UA-2017-01-19-001227-a,Управління освіти Подільської районної в м.Києві державної адміністрації,"згідно ЄЗС ДК 021:2015 – 55510000-8 ""Послуги їдальнь""(Оплата послуг з: лот 1- з гарячого харчування для учнів ЗНЗ; лот 2- з гарячого харчування для учнів Ліцею №100; лот 3- з гарячого харчування ...",964300,"згідно ЄЗС ДК 021:2015 – 55510000-8 ""Послуги їдальнь""(Оплата послуг з: лот 1- з гарячого харчування для учнів ЗНЗ; лот 2- з гарячого харчування для учнів Ліцею №100; лот 3- з гарячого харчування ...",лот 2- послуга з гарячого харчування для учнів Ліцею №100,Введенська 35,"вул.Введенська, 35",КМДА,37393777,Ліцей,False,False,False,№100№100№19
790,UA-2017-01-19-001227-a-L3,UA-2017-01-19-001227-a,Управління освіти Подільської районної в м.Києві державної адміністрації,"згідно ЄЗС ДК 021:2015 – 55510000-8 ""Послуги їдальнь""(Оплата послуг з: лот 1- з гарячого харчування для учнів ЗНЗ; лот 2- з гарячого харчування для учнів Ліцею №100; лот 3- з гарячого харчування ...",5

In [194]:
df_tenders_not_by_schools_numerated_all['count'] = df_tenders_not_by_schools_numerated_all['Номер'].str.count('№')

In [196]:
df_tenders_not_by_schools_numerated_all = df_tenders_not_by_schools_numerated_all[df_tenders_not_by_schools_numerated_all['count'] == 1]

In [197]:
df_tenders_not_by_schools_numerated_all

,ИдентификаторЛота,Идентификатор,Организатор,Тендер,СуммаЛота,ОписаниеТендера,Лот,Адрес поставки,АдресОрганизатора,Главный орган,IDOrganizator,Тип закладу,Тендер_check,ОписаниеТендера_check,Лот_check,Номер,count
159,UA-2016-12-14-000943-a-L1,UA-2016-12-14-000943-a,Управління освіти Солом'янської районної в місті Києві державної адміністрації,Капітальний ремонт місць загального користування в СЗШ №166,200000,Капітальний ремонт місць загального користування в СЗШ №166,Капітальний ремонт місць загального користування в СЗШ №166,"вул.Єреванська, 20","вулиця Пітерська, 12",КМДА,37485490,Загальноосвітня школа,False,False,False,№166,1
248,UA-2016-11-02-000583-a-L1,UA-2016-11-02-000583-a,Управління у справах фізичної культури і спорту Миколаївської міської ради,"капітальний ремонт системи автоматичної пожежної сигналізації та сповіщення про пожежу в приміщеннях дитячо-юнацької спортивної школи №1 за адресою: м. Миколаїв, вул. Театральна, 41 А",151705.2,додаток до річного плану,"капітальний ремонт системи автоматичної пожежної сигналізації та сповіщення про пожежу в приміщеннях дитячо-юнацької спортивної школи №1 за адресою: м. Миколаїв, вул. Театральна, 41 А","вул. Потьомкінська,95-А","вул. Потьомкінська,95-А",Миколаївська міська рада,2930482,Невідомо,False,False,False,№1,1
258,UA-2016-12-20-002973-b-L1,UA-2016-12-20-002973-b,Департамент житлово-комунального господарства Чернівецької міської ради,Капітальний ремонт світлофорного об'єкту на вул.Південно-Кільцевій - вул. С.Воробкевича - ЗНЗ №22 в м.Чернівцях,241312,NaN,Капітальний ремонт світлофорного об'єкту на вул.Південно-Кільцевій - вул. С.Воробкевича - ЗНЗ №22 в м.Чернівцях,Героїв Майдану 176,"вул. Героїв Майдану,176",Чернівецька міська рада,25082708,Невідомо,False,False,False,№22,1
286,UA-2016-09-19-000183-a-L1,UA-2016-09-19-000183-a,Управління освіти Солом'янської районної в місті Києві державної адміністрації,Поточний ремонт котельні дитячого садку №211,2990,Поточний ремонт котельні дитячого садку №211,Поточний ремонт котельні дитячого садку №211,"Вул. Леніна, 167","вулиця Пітерська, 12",КМДА,37485490,Невідомо,False,False,False,№211,1
297,UA-2016-12-29-000878-c-L1,UA-2016-12-29-000878-c,Управління освіти Солом'янської районної в місті Києві державної адміністрації,Поточний (аварійний) ремонт електрощитової з заміною лічильника в гімназії №177,59049.6,Поточний (аварійний) ремонт електрощитової з заміною лічильника в гімназії №177,Поточний (аварійний) ремонт електрощитової з заміною лічильника в гімназії №177,"вул. Курська, 12","вулиця Пітерська, 12",КМДА,37485490,Гімназія,False,False,False,№177,1
1006,UA-2017-04-28-001555-b-L1,UA-2017-04-28-001555-b,Управління освіти Солом'янської районної в місті Києві державної адміністрації,"ДСТУ Б.Д. 1.1-1:2013 Капітальний ремонт інженерних мереж в СЗШ № 12 за адресою: вул. Зелена, буд. 12 Солом’янського району м. Києва (за кодом CPV 45450000-6 Інші завершальні будівельні роботи)",306000,"ДСТУ Б.Д. 1.1-1:2013 Капітальний ремонт інженерних мереж в СЗШ № 12 за адресою: вул. Зелена, буд. 12 Солом’янського району м. Києва (за кодом CPV 45450000-6 Інші завершальні будівельні роботи)","ДСТУ Б.Д. 1.1-1:2013 Капітальний ремонт інженерних мереж в СЗШ № 12 за адресою: вул. Зелена, буд. 12 Солом’янського району м. Києва (за кодом CPV 45450000-6 Інші завершальні будівельні роботи)","вул. Зелена, буд. 12","вулиця Пітерська, 12",КМДА,37485490,Загальноосвітня школа,False,False,False,№ 12,1
1070,UA-2017-06-02-001017-a-L1,UA-2017-06-02-001017-a,ВК Управління освіти виконавчого комітету Полтавської міської ради,"Капітальний ремонт приміщення харчоблоку Полтавського СНВК № 45 Полтавської міської ради по вул. Грушевського, 17а",320383.2,NaN,"Капітальний ремонт приміщення харчоблоку Полтавського СНВК № 45 Полтавської міської ради по вул. Грушевського, 17а","вул. Грушевського, 17а","вул. Соборності, 36",Полтавська міська рада,2145725,Навчально-виховний комплекс (об'єднання),True,False,True,№ 45,1
1096,UA-2017-04-28-000337-c-L1,UA-2017-04-28-000337-c,Управління осв

TODO:
    залежність ОписаниеТендера і точність № в залежності від вижвання слова Додаток

##### Step 7.5: Найменування шкіл-не-замовників

In [165]:
columns_to_name = ['Тендер']

df_tenders_not_by_schools_named = naming_one_column(df_tenders_not_by_schools_not_numerated, columns_to_name, reg_quotes)

In [ ]:
def naming_one_column(given_df, columns_to_numerate, regex):
    for column in columns_to_numerate:
        given_df['Назва'] = given_df[column].str.findall(f"{regex}", flags = re.IGNORECASE).str[0].str[-1]
    return given_df

In [174]:
columns_to_name = ['Лот']

df_tenders_not_by_schools_named_2 = df_tenders_not_by_schools_named[df_tenders_not_by_schools_named['Назва'].isnull()] | (df_tenders_not_by_schools_named['Назва'] != '')]
df_tenders_not_by_schools_named_2 = naming_one_column(df_tenders_not_by_schools_named_2, columns_to_name, reg_quotes)

In [176]:
columns_to_name = ['ОписаниеТендера']

df_tenders_not_by_schools_named_3 = df_tenders_not_by_schools_named_2[(df_tenders_not_by_schools_named_2['Назва'].isnull()) | (df_tenders_not_by_schools_named_2['Назва'] != '')]
df_tenders_not_by_schools_named_3 = naming_one_column(df_tenders_not_by_schools_named_3, columns_to_name, reg_quotes)

In [178]:
df_tenders_not_by_schools_not_named = df_tenders_not_by_schools_named_3[(df_tenders_not_by_schools_named_3['Назва'].isnull()) | (df_tenders_not_by_schools_named_3['Назва'] == '')]

In [179]:
df_tenders_not_by_schools_named_3 = df_tenders_not_by_schools_named_3[(df_tenders_not_by_schools_named_3['Назва'].notnull()) | (df_tenders_not_by_schools_named_3['Назва'] != '')]

In [186]:
df_tenders_not_by_schools_named_3.shape
df_tenders_not_by_schools_named_4 = df_tenders_not_by_schools_named_3[df_tenders_not_by_schools_named_3['Назва'].notnull()]
df_tenders_not_by_schools_named_4 = df_tenders_not_by_schools_named_4[df_tenders_not_by_schools_named_4['Назва'] != '']

In [188]:
df_tenders_not_by_schools_named_4

,ИдентификаторЛота,Идентификатор,Организатор,Тендер,СуммаЛота,ОписаниеТендера,Лот,Адрес поставки,АдресОрганизатора,Главный орган,IDOrganizator,Тип закладу,Тендер_check,ОписаниеТендера_check,Лот_check,Номер,Назва
1171,UA-2017-11-28-004476-c-L1,UA-2017-11-28-004476-c,"КПНЗ ""Дитяча театральна школа м. Одеси""","Капітальний ремонт фасаду КПНЗ ""Дитяча театральна школа м.Одеси"" за адресою: м.Одеса, вул. Софіївська,24",975770.4,"Капітальний ремонт фасаду КПНЗ ""Дитяча театральна школа м.Одеси"" за адресою: м.Одеса, вул. Софіївська,24","Капітальний ремонт фасаду КПНЗ ""Дитяча театральна школа м.Одеси"" за адресою: м.Одеса, вул. Софіївська,24","вул.Софіївська,24","вул. Софіївська, 24",Одеська міська рада,13911602,Невідомо,False,False,False,,Дитяча театральна школа м.Одеси
1392,UA-2017-12-26-001838-a-L1,UA-2017-12-26-001838-a,Управління капітального будівництва Чернігівської міської ради,"Будівництво навчально-виховного комплексу ""Дошкільний навчальний заклад- загальноосвітній навчальний заклад І -ІІ ступенів"" у мікрорайоні ""Бобровиця"" на 345 місць (ДНЗ - 75 місць, ЗНЗ - 270 місць)...",138902.4,"Будівництво навчально-виховного комплексу ""Дошкільний навчальний заклад- загальноосвітній навчальний заклад І -ІІ ступенів"" у мікрорайоні ""Бобровиця"" на 345 місць (ДНЗ - 75 місць, ЗНЗ - 270 місць)...","Будівництво навчально-виховного комплексу ""Дошкільний навчальний заклад- загальноосвітній навчальний заклад І -ІІ ступенів"" у мікрорайоні ""Бобровиця"" на 345 місць (ДНЗ - 75 місць, ЗНЗ - 270 місць)...","вул. Івана Мазепи, 19","вул. Івана Мазепи, 19, к. 216",Чернігівська міська рада,5517729,Навчально-виховний комплекс (об'єднання),True,True,True,,Дошкільний навчальний заклад- загальноосвітній навчальний заклад І -ІІ ступенів
2503,UA-2017-02-14-000264-a-L1,UA-2017-02-14-000264-a,Управління освіти адміністрації Новобаварського району Харківської міської ради,"Послуги з технічної підтримки програмного комплексу ""КУРС"" для використання в загальноосвітніх навчальних закладах",13240,"Послуги з технічної підтримки програмного комплексу ""КУРС"" для використання в загальноосвітніх навчальних закладах","Послуги з технічної підтримки програмного комплексу ""КУРС"" для використання в загальноосвітніх навчальних закладах",ЗНЗ Новобаварського району м.Харкова,"вул.Ярославська,буд.21",Харківська міська рада,2146311,Загальноосвітня школа,False,False,False,,КУРС
5348,UA-2017-03-16-001871-b-L1,UA-2017-03-16-001871-b,Управління освіти Шевченківської районної в місті Києві державної адміністрації,"Поточний ремонт приміщень пральні НВК ""Кияночка"", м. Київ, вул. Шамрила, 7-а",350000,"Поточний ремонт приміщень пральні НВК ""Кияночка"", м. Київ, вул. Шамрила, 7-а","Поточний ремонт приміщень пральні НВК ""Кияночка"", м. Київ, вул. Шамрила, 7-а","Шамрила, 7-А","Зоологічна, 6-а",КМДА,37470086,Навчально-виховний комплекс (об'єднання),True,True,True,,Кияночка
5393,UA-2017-10-04-000332-b-L1,UA-2017-10-04-000332-b,Управління капітального будівництва Чернігівської міської ради,"Реконструкція стадіону ""Локомотив"" на земельній ділянці за згодою її користувача - комунального позашкільного навчального закладу ""Спеціалізована дитячо-юнацька школа олімпійського резерву з футбо...",140526.56,"Реконструкція стадіону ""Локомотив"" на земельній ділянці за згодою її користувача - комунального позашкільного навчального закладу ""Спеціалізована дитячо-юнацька школа олімпійського резерву з футбо...","Реконструкція стадіону ""Локомотив"" на земельній ділянці за згодою її користувача - комунального позашкільного навчального закладу ""Спеціалізована дитячо-юнацька школа олімпійського резерву з футбо...","вул.Івана Мазепи, 19","вул. Івана Мазепи, 19, к. 216",Чернігівська міська рада,5517729,Спеціалізована школа,False,False,False,,Локомотив
6261,UA-2017-08-03-000719-c-L1,UA-2017-08-03-000719-c,"Департамент у справах сім'ї, молоді та спорту Харківської міської ради","Послуги з оренди апаратури для проведення заходу в рамках проведення ""Школи волонтерів"" з 4 по 25 серпня 2017 

In [180]:
df_tenders_not_by_schools_not_named.shape

(4038, 17)

In [ ]:
training_data_columns

In [ ]:
df_tenders_not_by_schools_classified

In [ ]:
step_mark = 'step4'
action_mark = 'naming'

columns_to_classify_step4_tenders_by_schools = ['Организатор']
df_step4_tenders_by_schools = df_tenders_by_schools_concatenated.copy()


df_step4 = df_step3.copy()

In [198]:
def naming(given_df, columns_to_classify, regex):
    for column in columns_to_classify:
        given_df[column + '_' + step_mark + '_' + action_mark] = given_df[column].str.findall(f"{regex}", 
                                                                                              flags = re.IGNORECASE).str[0].str[-1]
    return given_df

In [ ]:
XXX = naming(df_tenders_not_by_schools_classified, columns_to_classify_step4_tenders_by_schools, reg_quotes)
df_step4_tenders_by_schools_filtered.sample(10)[['Организатор', 'Организатор_step4_naming']]

In [200]:
columns_to_classify_step4 = ['Тендер', 'ОписаниеТендера', 'Лот']
step_mark = 'step4'
action_mark = 'naming'

df_step4_filtered = naming(df_tenders_not_by_schools_classified, columns_to_classify_step4, reg_quotes)
df_step4_filtered.sample(10)

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,ИдентификаторЛота,Идентификатор,Организатор,Тендер,СуммаЛота,ОписаниеТендера,Лот,Адрес поставки,АдресОрганизатора,Главный орган,IDOrganizator,Тип закладу,Тендер_check,ОписаниеТендера_check,Лот_check,Номер,Тендер_step4_naming,ОписаниеТендера_step4_naming,Лот_step4_naming
180105,UA-2018-08-03-001684-b-L1,UA-2018-08-03-001684-b,ДЕПАРТАМЕНТ ГУМАНІТАРНОЇ ПОЛІТИКИ ДНІПРОВСЬКОЇ МІСЬКОЇ РАДИ,"Поточний ремонт фасаду Комунального закладу освіти ""Середня загальноосвітня школа №51"" Дніпровської міської ради, за адресою: вул. Алтайська, 4а, м. Дніпро",199988,NaN,"Поточний ремонт фасаду Комунального закладу освіти ""Середня загальноосвітня школа №51"" Дніпровської міської ради, за адресою: вул. Алтайська, 4а, м. Дніпро","вул. Алтайська, 4а","просп. Д.Яворницького 75,А",Дніпровська міська рада,40506248,Загальноосвітня школа,False,False,False,№51,Середня загальноосвітня школа №51,NaN,Середня загальноосвітня школа №51
188953,UA-2018-11-21-002085-a-L1,UA-2018-11-21-002085-a,Управління освіти Дарницької районної в місті Києві державної адміністрації,ДК 021: 2015 код 71320000-7 «Послуги з інженерного проектування» (послуги з розробки проектно-кошторисної документації стадії «РП» з улаштування систем блискавкозахисту в дошкільних навчальних за...,70700,ДК 021: 2015 код 71320000-7 «Послуги з інженерного проектування» (послуги з розробки проектно-кошторисної документації стадії «РП» з улаштування систем блискавкозахисту в дошкільних навчальних за...,ДК 021: 2015 код 71320000-7 «Послуги з інженерного проектування» (послуги з розробки проектно-кошторисної документації стадії «РП» з улаштування систем блискавкозахисту в дошкільних навчальних за...,"Харківське шосе, 168к","вул. Харківське шосе, 168-к",КМДА,37448113,ДНЗ,False,False,False,№31№89№99№113№133№126№148,,,
239757,UA-2018-01-15-000502-a-L3,UA-2018-01-15-000502-a,"КНП""ЦПМСД№1 Дніпровського району м.Києва""","пара, гаряча вода та пов""язана продукція (теплова енергія у гарячій воді)",91243.9,"Закупівля теплової енергії у гарячій воді проводиться по 6 лотам .Обсяг закупівлі - 1091 Гкал. Загальна сума закупівлі -1 816 023,34 грн. Учасник повинен підтримувати середньодобову температуру т...",Лот 3- теплова енергія у гарячій воді,"вул. Курнатовського, 5","вулиця Петра Запорожця,26",КМДА,26188946,Загальноосвітня школа,False,False,False,,NaN,Київенерго,NaN
433652,UA-2019-09-06-001282-b-L1,UA-2019-09-06-001282-b,Управління освіти Херсонської міської ради,"Ноутбуки, планшети, моноблоки, комп'ютерні тифлокомплекси, комп'ютера у комплекті",27000,NaN,Лот №1 ноутбуки для інклюзивних класів НВК №9,За адресами закладів управління освіти Херсонської міської ради,"вул. Воронцовська, 15-А",Херсонська міська рада,2146713,Навчально-виховний комплекс (об'єднання),False,False,True,,,NaN,NaN
524771,UA-2019-03-11-000514-c-L1,UA-2019-03-11-000514-c,ВК Управління освіти виконавчого комітету Полтавської міської ради,"Поточний ремонт внутрішньої каналізаційної мережі в ДНЗ № 56 за адресою: м. Полтава, вул. Велика, 2-а",5094.67,NaN,"Поточний ремонт внутрішньої каналізаційної мережі в ДНЗ № 56 за адресою: м. Полтава, вул. Велика, 2-а","вул. Велика, 2-а","вул. Соборності, 36",Полтавська міська рада,2145725,ДНЗ,False,False,False,№ 56,NaN,NaN,NaN
533529,UA-2019-03-25-001407-b-L1,UA-2019-03-25-001407-b,УПРАВЛІННЯ КАПІТАЛЬНОГО БУДІВНИЦТВА ПОЛТАВСЬКОГО МІСЬКВИКОНКОМУ,"""Будівництво будівлі харчоблоку та пральні Комунального закладу ""Полтавський спеціалізований навчально-виховний комплекс № 26 ""Перші кроки"" Полтавської міської ради Полтавської області"" по вул. С...",12239332,NaN,"""Будівництво будівлі харчоблоку та пральні Комунального закладу ""Полтавський спеціалізований навчально-виховний комплекс № 26 ""Перші кроки"" Полтавської міської ради Полтавської області"" по вул. С...","Стрітенська, 20","вул. Соборності, 36",Полтавська міська рада,4057600,Навчально-виховний комплекс (об'єднання),True,False,True,№ 26,Будівництво будівлі харчоблоку та пральні Комунального закладу,NaN,Будівництво будівлі харчоблоку та пральні Комуна

In [ ]:
##### Step 2: Виділення закупівель шкіл-замовників за допомогою пошуку ключових слів в колонці "Организатор" 

##### Step 2: Виділення закупівель шкіл-замовників за допомогою пошуку ключових слів в колонці "Организатор" 

In [ ]:
step_mark = 'step0'
action_mark = 'check'
number_0 = '0'
columns_to_check_step0 = ['Организатор']

# Перший спосіб відібрати навчальні заклади, які є замовниками: пошук за ЄДРПОУ в полі "IDOrganizator"
df_tenders_by_schools_by_edrpou = filter_dataframe(df_raw, positive_edrpous(), ['IDOrganizator'])

# Створити базу для закупівель навчальних закладів, які НЕ є замовниками
df_merged_left = pandas.merge(df_raw, df_tenders_by_schools_by_edrpou, on='ИдентификаторЛота', 
                              how = 'left', indicator = True, suffixes = ('','_temp'))
df_undefined_0 = df_merged_left[df_merged_left._merge=='left_only']
df_undefined_0 = df_undefined_0[training_data_columns]

# Другий спосіб відібрати навчальні заклади, які є замовниками: пошук за ключовими словами в полі "Организатор"
df_step0 = df_undefined_0.copy()

for column in columns_to_check_step0:
    df_step0 = step_check(df_step0, column, number_0)
    
df_tenders_by_schools_by_keywords_0, df_undefined_0 = separate_positives_and_negatives_ogranizator(df_step0, step_mark, action_mark)
print(f"{step_mark}:", "df_tenders_by_schools_by_keywords:", df_tenders_by_schools_by_keywords_0.shape[0])

#### Concatenating 2 dataframes: df_tenders_by_schools_concatenated

In [ ]:
# df_tenders_by_schools_concatenated - 
# Об'єднання двох баз закупівель навчальних закладів, які самостійно купують через ProZorro

frames_tenders_by_schools = [df_tenders_by_schools_by_edrpou, df_tenders_by_schools_by_keywords_0]
df_tenders_by_schools_concatenated = pandas.concat(frames_tenders_by_schools)
len_df = df_tenders_by_schools_concatenated.shape[0]
df_tenders_by_schools_concatenated.drop_duplicates(subset = 'ИдентификаторЛота', inplace = True)
print("Removed duplicates:", len_df - df_tenders_by_schools_concatenated.shape[0])

### Step 1: 
#### Створити базу закупівель навчальних закладів, які купують у ProZorro через посередників: управління освіти, міністерства

In [ ]:
# TODO Step 1:
# ПЕРЕВІРИТИ, ЩО ЗНАХОДИТЬ В КОЛОНЦІ "АДЕРСА ПОСТАВКИ"

"""
За результатами точності визначення закупівель вирішити чи фільтрувати за цією колонкою.
Проблема: в полі адреса часто вказують ключові слова, які ніби можуть вказувати на приналежність
закупівлі до шкільних. Наприклад, вул. Шкільна, Гімназійна і т.п.
"""

In [ ]:
step_mark = 'step1'
action_mark = 'check'
number_1 = '1'
columns_to_check_step1 = ['Тендер', 'ОписаниеТендера', 'Лот']
df_step1 = df_undefined_0.copy()

for column in columns_to_check_step1:
    df_step1 = step_check(df_step1, column, number_1)

df_tenders_by_schools_by_keywords_1, df_undefined_1 = separate_positives_and_negatives(df_step1, step_mark, action_mark)
df_tenders_by_schools_by_keywords_1 = df_tenders_by_schools_by_keywords_1[training_data_columns]
df_not_school_tenders = df_undefined_1[training_data_columns]
print(f"{step_mark}:", "df_tenders_by_schools_by_keywords_1:", df_tenders_by_schools_by_keywords_1.shape[0])

In [ ]:
# TODO:
#    Inspect df_not_school_tenders.csv
#   df_not_school_tenders.to_csv('df_not_school_tenders.csv');

### Step 2:
#### Класифікація навчальних закладів, які самостійно купують через ProZorro

In [ ]:
# TODO:
# Визначити пріоритетність фільтрів для визначення категорії. Наприклад, якщо в назві є НВК і дитячий садок.

def classification(given_df, columns_to_classify, filter_list, filter_name):
    for column in columns_to_classify:
        given_df[column + '_' + step_mark + '_' + action_mark] = given_df[column].str.contains('|'.join(filter_list), 
                                                                                               case = False, 
                                                                                               na = False)
    for i in range(given_df.shape[0]):
        for column in columns_to_classify:
            if given_df.loc[given_df.index[i], column + '_' + step_mark + '_classification'].any():
                given_df.loc[given_df.index[i], 'Тип закладу'] = filter_name
                break;
    for column in columns_to_classify:
        given_df = clear_redundant_columns(given_df, step_mark, action_mark, column)
    return given_df

In [ ]:
step_mark = 'step2'
action_mark = 'classification'
columns_to_classify_step2 = ['Организатор']
df_step2 = df_tenders_by_schools_concatenated.copy()

In [ ]:
df_step2_random = df_step2.copy().sample(1000)

#### Filtering

In [ ]:
for idx in range(len(filter_priority)):
    df_step2_random = classification(df_step2_random, columns_to_classify_step2, filter_priority[idx], filter_name_priority[idx])

#### Checking results

In [ ]:
df_step2_random[df_step2_random['Тип закладу'].notnull()].shape

In [ ]:
print(df_step2_random[df_step2_random['Тип закладу'].isnull()].shape)

columns_to_classify_step2_temp = columns_to_classify_step2.copy()
columns_to_classify_step2_temp.append('Тип закладу')

df_step2_random[df_step2_random['Тип закладу'].isnull()][columns_to_classify_step2_temp]

In [ ]:
df_step2_random[df_step2_random['Тип закладу'] == 'ДНЗ'][columns_to_classify_step2_temp].sample(10)

### Step 3:
#### Класифікація навчальних закладів, для яких купують управління освіти, міністерства чи інші органи влади

In [ ]:
# TODO:
# ПЕРЕВІРИТИ, ЩО ЗНАХОДИТЬ В КОЛОНЦІ "ОРГАНИЗАТОР"

In [ ]:
step_mark = 'step3'
action_mark = 'classification'
columns_to_classify_step3 = ['Тендер', 'ОписаниеТендера', 'Лот', 'Адрес поставки']
df_step3 = df_tenders_by_schools_by_keywords_1.copy()

In [ ]:
df_step3_random = df_step3.copy().sample(1000)

#### Filtering

In [ ]:
for idx in range(len(filter_priority)):
    df_step3_random = classification(df_step3_random, columns_to_classify_step3, 
                                    filter_priority[idx], filter_name_priority[idx])

#### Checking results

In [ ]:
columns_to_classify_step3_temp = columns_to_classify_step3.copy()
columns_to_classify_step3_temp.append('Тип закладу')

In [ ]:
df_step3_random['Тип закладу'].value_counts()

In [ ]:
df_step3_random[df_step3_random['Тип закладу'].notnull()].shape

In [ ]:
print(df_step3_random[df_step3_random['Тип закладу'].isnull()].shape)
df_step3_random[df_step3_random['Тип закладу'].isnull()][columns_to_classify_step3_temp]

### Step 4:
####  присвоєння навчальним закладам власних назв

In [ ]:
step_mark = 'step4'
action_mark = 'naming'

columns_to_classify_step4_tenders_by_schools = ['Организатор']
df_step4_tenders_by_schools = df_tenders_by_schools_concatenated.copy()

columns_to_classify_step4 = ['Тендер', 'ОписаниеТендера', 'Лот', 'Адрес поставки']
df_step4 = df_step3.copy()

In [ ]:
def naming(given_df, columns_to_classify, regex):
    for column in columns_to_classify:
        given_df[column + '_' + step_mark + '_' + action_mark] = given_df[column].str.findall(f"{regex}", 
                                                                                              flags = re.IGNORECASE).str[0].str[-1]
    return given_df

In [ ]:
df_step4_tenders_by_schools_filtered = naming(df_step4_tenders_by_schools, columns_to_classify_step4_tenders_by_schools, reg_quotes)
df_step4_tenders_by_schools_filtered.sample(10)[['Организатор', 'Организатор_step4_naming']]

In [ ]:
df_step4_filtered = naming(df_step4, columns_to_classify_step4, reg_quotes)
df_step4_filtered.sample(10)

### Step 5:
####  присвоєння номерів

In [ ]:
step_mark = 'step5'
action_mark = 'numeration'

columns_to_classify_step5_tenders_by_schools = ['Организатор']
df_step4_tenders_by_schools = df_tenders_by_schools_concatenated.copy()

columns_to_classify_step5 = ['Тендер', 'ОписаниеТендера', 'Лот', 'Адрес поставки']
df_step5 = df_step3.copy()

In [ ]:
def numeration(given_df, columns_to_classify, regex):
    for column in columns_to_classify:
        given_df[column + '_' + step_mark + '_' + action_mark] = given_df[column].str.findall(f"{regex}", 
                                                                                              flags = re.IGNORECASE)
    return given_df

In [ ]:
df_step5_tenders_by_schools_filtered = numeration(df_step4_tenders_by_schools, columns_to_classify_step4_tenders_by_schools, reg_number)
df_step5_tenders_by_schools_filtered

In [ ]:
df_step5_filtered = numeration(df_step4, columns_to_classify_step4, reg_number)
df_step5_filtered.sample(5)

### Step 6:
####  зведення в єдину базу

### Optional step

#### За ЄДРПОУ видалити рядки із закупівлями закладів, які не є закладами загальної середньої чи дошкільної освіти

In [ ]:
# TODO: 
# До переліку "негативних" ЄДРПОУ включені управління освіти. 
# Тому заклади, де є "управління освіти" в полі організатор, не видаляти

In [ ]:
# filter_dataframe_inspect_before_drop(df_step1_positives, negative_edrpou(), ['IDOrganizator'])['IDOrganizator'].unique().tolist()

In [ ]:
#filter_dataframe_inspect_before_drop_step_0['Организатор'].unique().tolist()

In [ ]:
# filter_dataframe_inspect_before_drop(df_step1_positives, filter_poza, ['Организатор'])['Организатор'].unique().tolist()

In [ ]:
#df_step2_positives = filter_dataframe_drop(df_step1_positives, negative_edrpou(), ['IDOrganizator'])
#df_step2_positives = filter_dataframe_drop(df_step2_positives, filter_poza, ['Организатор'])
#print('Залишилось закупівель навчальних закладів, які не є замовниками:', df_step2_positives.shape[0])